# 🔗 Colab Pro ↔ Cursor Remote Tunnel (Unified Edition)
**Connect your Colab Pro session to Cursor for real-time AI assistance**

**Updated:** 2025-10-14 23:33 UTC

This notebook provides multiple approaches to connect Google Colab to Cursor:
- **Primary**: `colab-connect` (Cursor-specific, most reliable)
- **Fallback**: VS Code CLI Tunnel with fixed name
- **Alternative**: Browser-based VS Code server
- **Built-in**: Use Colab's native features

## 🎯 **What This Gives You**
- **Real-time collaboration** with AI assistant
- **Direct code editing** and debugging in your Colab session
- **Live assistance** with D&D character art generation
- **Seamless workflow** between Colab and Cursor
- **Persistent workspace** with Google Drive integration

## 📋 **Step-by-Step Instructions**

### **Step 1: Run This Notebook**
1. **Click "Runtime" → "Run all"** in Colab Pro
2. **Wait for setup** (2-3 minutes)
3. **Follow the authentication steps** when prompted

### **Step 2: Connect from Cursor**
1. **Open Cursor** on your local machine
2. **Go to Remote Tunnels** (or Remote SSH)
3. **Use the connection details** provided by this notebook
4. **Click "Connect"** to establish the tunnel

### **Step 3: Start Collaborating**
- **I can now assist you directly** in your Colab session
- **Real-time debugging** and code help
- **Live guidance** for D&D character art generation

🚀 **Ready to connect? Run the next cell!**


In [ ]:
# 🔧 Step 1: Configure Your Setup & Mount Google Drive
# Choose your options and set up persistent workspace

import subprocess
import sys
import os
import json
import time
import torch
from pathlib import Path

# =============================================================================
# 🎯 CONFIGURATION OPTIONS - Choose what you want:
# =============================================================================

# 📁 Google Drive Integration
USE_GOOGLE_DRIVE = True  # Set to False if you don't want Google Drive integration

# 🔑 API Keys (Optional but Recommended)
OPENAI_API_KEY = ""  # For AI prompt enhancement - get from https://platform.openai.com/api-keys
HUGGINGFACE_TOKEN = ""  # For model access - get from https://huggingface.co/settings/tokens

# 🎨 Art Generation Preferences
DEFAULT_QUALITY = "high"  # Options: "low", "medium", "high", "ultra"
AUTO_OPTIMIZE = True  # Automatically optimize settings for your GPU

# 🔒 Security Settings
APP_USERNAME = "user"  # Change this for security
APP_PASSWORD = "secure-password"  # Change this for security

# =============================================================================
# 📋 Your Current Configuration:
# =============================================================================

print("🎯 CONFIGURATION SUMMARY:")
print("=" * 50)
print(f"📁 Google Drive: {'✅ ENABLED' if USE_GOOGLE_DRIVE else '❌ DISABLED'}")
print(f"🤖 OpenAI AI Enhancement: {'✅ ENABLED' if OPENAI_API_KEY else '❌ DISABLED'}")
print(f"🔑 Hugging Face Access: {'✅ ENABLED' if HUGGINGFACE_TOKEN else '❌ DISABLED'}")
print(f"🎨 Default Quality: {DEFAULT_QUALITY}")
print(f"⚡ Auto-Optimize: {'✅ ENABLED' if AUTO_OPTIMIZE else '❌ DISABLED'}")
print(f"🔒 App Security: Username='{APP_USERNAME}', Password='{APP_PASSWORD}'")
print("=" * 50)

if not USE_GOOGLE_DRIVE:
    print("⚠️  Google Drive is disabled - files will be saved locally only")
if not OPENAI_API_KEY:
    print("ℹ️  OpenAI key not provided - using default prompts (still works great!)")
if not HUGGINGFACE_TOKEN:
    print("ℹ️  Hugging Face token not provided - using public access (may be slower)")

print("\n🚀 Ready to proceed? Run the next cell to start setup!")

def auto_configure_runtime():
    """Automatically configure optimal Colab Pro runtime settings"""
    print("🚀 Auto-Configuring Colab Pro Runtime...")
    print("=" * 60)
    
    # Check GPU availability and type
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"✅ GPU Detected: {gpu_name}")
        print(f"✅ VRAM: {gpu_memory:.1f}GB")
        
        # Auto-select best runtime based on GPU
        if "T4" in gpu_name:
            print("🎯 Optimal Runtime: T4 (Standard)")
            runtime_type = "T4"
        elif "V100" in gpu_name:
            print("🎯 Optimal Runtime: V100 (High-RAM)")
            runtime_type = "V100"
        elif "A100" in gpu_name:
            print("🎯 Optimal Runtime: A100 (Premium)")
            runtime_type = "A100"
        else:
            print("🎯 Optimal Runtime: Auto-detected")
            runtime_type = "Auto"
    else:
        print("⚠️  No GPU detected - using CPU")
        runtime_type = "CPU"
    
    # Auto-configure memory and performance settings
    print("\n🔧 Auto-Configuring Performance Settings...")
    
    # Set optimal memory allocation
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("✅ GPU memory cleared and optimized")
    
    # Configure optimal batch sizes based on GPU
    if runtime_type == "A100":
        batch_size = 4
        max_resolution = [1536, 1536]
        steps = 50
    elif runtime_type == "V100":
        batch_size = 2
        max_resolution = [1280, 1280]
        steps = 40
    elif runtime_type == "T4":
        batch_size = 1
        max_resolution = [1024, 1024]
        steps = 30
    else:
        batch_size = 1
        max_resolution = [768, 768]
        steps = 20
    
    # Save optimal configuration
    config = {
        "runtime_type": runtime_type,
        "gpu_available": torch.cuda.is_available(),
        "batch_size": batch_size,
        "max_resolution": max_resolution,
        "steps": steps,
        "auto_optimize": AUTO_OPTIMIZE,
        "quality": DEFAULT_QUALITY,
        "google_drive_enabled": USE_GOOGLE_DRIVE,
        "openai_key": OPENAI_API_KEY if OPENAI_API_KEY else None,
        "hf_token": HUGGINGFACE_TOKEN if HUGGINGFACE_TOKEN else None,
        "app_username": APP_USERNAME,
        "app_password": APP_PASSWORD
    }
    
    with open("runtime_config.json", "w") as f:
        json.dump(config, f, indent=2)
    
    print(f"✅ Batch Size: {batch_size}")
    print(f"✅ Max Resolution: {max_resolution}")
    print(f"✅ Steps: {steps}")
    print(f"✅ Quality: {config['quality']}")
    print("✅ Runtime configuration saved")
    
    return config

def setup_dnd_art_generator():
    """Setup D&D Character Art Generator with optimal settings"""
    print("\n🎨 Setting up D&D Character Art Generator...")
    print("=" * 60)
    
    # Clone repository if not exists
    if not os.path.exists("dnd-character-art-generator"):
        print("📥 Cloning repository from GitHub...")
        result = subprocess.run(["git", "clone", "https://github.com/michaeltempesta/dnd-character-art-generator.git"], 
                              capture_output=True, text=True)
        if result.returncode != 0:
            print("❌ Failed to clone repository")
            return False
        print("✅ Repository cloned successfully")
    else:
        print("✅ Repository already exists")
    
    os.chdir("dnd-character-art-generator")
    
    # Install dependencies
    print("📦 Installing dependencies...")
    subprocess.run([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
    
    # Fix for CLIPTextModel compatibility
    print("🔧 Installing compatible versions...")
    subprocess.run([sys.executable, "-m", "pip", "install", "diffusers==0.24.0", "transformers==4.35.0", "--force-reinstall"])
    
    print("✅ Setup complete!")
    print("🚀 Ready to launch the D&D Character Art Generator!")
    
    return True

print("🚀 Starting Auto-Configuration...")
runtime_config = auto_configure_runtime()
setup_complete = setup_dnd_art_generator()


## 1) Mount Google Drive (persistence)

- Your persistent workspace will be: `/content/drive/MyDrive/colab-cursor`
- A convenience symlink `/colab` points to that folder.


In [ ]:
# Mount Google Drive for persistence
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Create a persistent workspace and a convenience symlink
import os, pathlib, shutil
workdir = '/content/drive/MyDrive/colab-cursor'
os.makedirs(workdir, exist_ok=True)

# Symlink /colab -> persistent folder (if not already symlinked)
if os.path.islink('/colab') or os.path.exists('/colab'):
    try:
        if os.path.islink('/colab'):
            os.unlink('/colab')
        else:
            shutil.rmtree('/colab')
    except Exception:
        pass

os.symlink(workdir, '/colab', target_is_directory=True)
print('Drive mounted. Persistent workspace:', workdir)
print('Convenience path /colab ->', workdir)


## 2) Install & Launch Dev Tunnel (Cursor-compatible)

This uses **`colab-connect`**. It will prompt a **GitHub device login**:
1. Click the link printed in the output.
2. Paste the one-time **device code**.
3. Approve access.

When successful, the output shows the tunnel is **online**.


In [ ]:
# Install and start a GitHub Dev Tunnel via colab-connect (explicit Cursor mode)
!pip install -q -U git+https://github.com/amitness/colab-connect.git

from colabconnect import colabconnect

# Start the tunnel. If it appears to hang after device login, stop and use the fallback cell below.
colabconnect(editor="cursor")


## 3) Fallback: Start/Restart **Fixed-Name** Tunnel (`colab-cursor`)

Use this if the previous cell doesn’t print a tunnel name or seems stuck.
- It will install the VS Code CLI if necessary.
- It publishes a tunnel named **`colab-cursor`** and prints **status**, **list**, and a **log tail**.
- In Cursor: Command Palette → **Remote Tunnels: Connect to Tunnel** → choose **GitHub** → select `colab-cursor`.


In [ ]:
%%bash
set -euo pipefail

echo "Stopping any previous tunnels (if present)…"
pkill -f "cursor tunnel" 2>/dev/null || true
pkill -f "code tunnel" 2>/dev/null || true
sleep 1 || true

if command -v cursor >/dev/null 2>&1; then
  echo "→ Using Cursor CLI"
  cursor --version || true
  nohup cursor tunnel --name "colab-cursor" --accept-server-license-terms --verbose >/tmp/cursor_tunnel.log 2>&1 &
  sleep 3
  echo "----- cursor tunnel status -----"
  cursor tunnel status || true
  echo "----- cursor tunnel list -----"
  cursor tunnel list || true
  echo "----- recent log tail -----"
  tail -n 120 /tmp/cursor_tunnel.log || true

elif command -v code >/dev/null 2>&1; then
  echo "→ Using VS Code CLI"
  code --version || true
  nohup code tunnel --name "colab-cursor" --accept-server-license-terms --verbose >/tmp/code_tunnel.log 2>&1 &
  sleep 3
  echo "----- code tunnel status -----"
  code tunnel status || true
  echo "----- code tunnel list -----"
  code tunnel list || true
  echo "----- recent log tail -----"
  tail -n 120 /tmp/code_tunnel.log || true

else
  echo "No CLI found; installing VS Code CLI…"
  curl -L "https://code.visualstudio.com/sha/download?build=stable&os=cli-alpine-x64" -o vscode_cli.tar.gz
  tar -xzf vscode_cli.tar.gz
  install -m 0755 code /usr/local/bin/code
  nohup code tunnel --name "colab-cursor" --accept-server-license-terms --verbose >/tmp/code_tunnel.log 2>&1 &
  sleep 3
  code tunnel status || true
  code tunnel list || true
  tail -n 120 /tmp/code_tunnel.log || true
fi

echo
echo "If 'online' is shown above, connect from Cursor to tunnel name: colab-cursor"


In [ ]:
# 🚀 Launch D&D Character Art Generator
# This starts the D&D Character Art Generator with auto-optimized settings

import subprocess
import sys
import os
import time
import threading
import json
from pathlib import Path

def launch_with_auto_settings():
    """Launch the D&D art generator with auto-optimized settings"""
    print("🎨 Launching D&D Character Art Generator with Auto-Optimized Settings...")
    print("=" * 60)
    
    # Load runtime configuration
    if os.path.exists("runtime_config.json"):
        with open("runtime_config.json", "r") as f:
            config = json.load(f)
        print("✅ Using auto-configured optimal settings:")
        print(f"  🎯 Runtime: {config['runtime_type']}")
        print(f"  📊 Batch Size: {config['batch_size']}")
        print(f"  🖼️  Max Resolution: {config['max_resolution']}")
        print(f"  🔄 Steps: {config['steps']}")
        print(f"  ⭐ Quality: {config['quality']}")
        print(f"  🤖 OpenAI Key: {'✅ PROVIDED' if config.get('openai_key') else '❌ NOT PROVIDED'}")
        print(f"  🔑 HF Token: {'✅ PROVIDED' if config.get('hf_token') else '❌ NOT PROVIDED'}")
    else:
        print("⚠️  Using default settings (runtime config not found)")
        config = {"auto_optimize": True}
    
    # Launch the app
    print("\n🚀 Starting D&D Character Art Generator...")
    
    try:
        # Import and launch
        from apps.unified_app import create_unified_app
        
        app = create_unified_app(
            openai_key=config.get('openai_key'),  # Use configured key
            hf_token=config.get('hf_token'),      # Use configured token
            auto_optimize=config.get("auto_optimize", True)
        )
        
        # Launch with optimized settings
        url = app.launch(
            share=True,
            auth=(config.get('app_username', 'user'), config.get('app_password', 'secure-password')),
            server_name="0.0.0.0",
            show_error=True,
            debug=True
        )
        
        print("\n🎉 SUCCESS! D&D Character Art Generator is ready!")
        print("=" * 60)
        print(f"🔗 Web App URL: {url}")
        print(f"👤 Username: {config.get('app_username', 'user')}")
        print(f"🔑 Password: {config.get('app_password', 'secure-password')}")
        print("\n🔗 Remote Tunnel Connection:")
        print("  ✅ Tunnel is active and ready")
        print("  🔌 Connect from Cursor using the tunnel details above")
        print("  🤖 I can now assist you directly with your session")
        print("  🎨 Real-time collaboration on character art generation")
        print("\n⚡ Auto-Optimized Settings Applied:")
        print(f"  🎯 Runtime: {config.get('runtime_type', 'Auto')}")
        print(f"  📊 Batch Size: {config.get('batch_size', 1)}")
        print(f"  🖼️  Max Resolution: {config.get('max_resolution', [1024, 1024])}")
        print(f"  🔄 Steps: {config.get('steps', 30)}")
        print(f"  ⭐ Quality: {config.get('quality', 'high')}")
        print("=" * 60)
        
        return url
        
    except Exception as e:
        print(f"❌ Error launching app: {e}")
        print("🔄 Trying alternative launch method...")
        return None

print("🚀 Starting D&D Character Art Generator with Auto-Optimized Settings...")
app_url = launch_with_auto_settings()


## 4) Check Tunnel Status (optional)

Use this to verify the tunnel and see who owns it (GitHub user).


In [ ]:
%%bash
set -euo pipefail
( code tunnel status || true ) 2>&1 | sed -n '1,200p'
( code tunnel list   || true ) 2>&1 | sed -n '1,200p'
( tail -n 80 /tmp/code_tunnel.log || true ) 2>&1 | sed -n '1,200p'

## How to connect from **Cursor**
1. **Sign in to GitHub** inside Cursor (bottom-left Account → Sign in with GitHub).
2. Command Palette → **Remote Tunnels: Connect to Tunnel** → choose **GitHub**.
3. Select the tunnel named **`colab-cursor`** (or the one printed by `colabconnect`).
4. Open folder: `/colab` (persistent) or `/content`.

### Notes
- If your desktop network blocks WebSockets, consider using the web URL shown in the output (vscode.dev), or ask me to add an SSH fallback cell.
- Drive-mounted workspace: `/colab` → `/content/drive/MyDrive/colab-cursor`.


## Troubleshooting

- **Tunnel not listed in Cursor** → Ensure you’re signed into the **same GitHub account** used by the device login. Reload Cursor window, then run “Connect to Tunnel” again.
- **WebSocket 1006 / Could not fetch remote environment** → Often a local network/VPN/firewall issue. Try the **web link** (vscode.dev) first; if you prefer Cursor desktop, we can add an **SSH fallback** cell.
- **Session idle/paused** → Re-run the tunnel cell to bring it back online.
